# Main script

In [1]:
import numba

numba.__version__

'0.58.1'

In [2]:
import scipy

scipy.__version__

'1.15.2'

In [3]:
import joblib

joblib.__version__

'1.2.0'

In [4]:
import os
import json
import zipfile
import warnings
import itertools
import numpy as np
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from numpy import arange

In [5]:
from pNeuma_simulator.simulate import batch
from pNeuma_simulator.results import zipdir

In [6]:
warnings.filterwarnings("ignore")

In [7]:
path = "./output/"
os.makedirs(path, exist_ok=True)
# Combinatorial configurations
n_veh = 8
scale = 2
l_cars = scale * arange(1, n_veh, 1)
l_moto = scale * arange(0, n_veh - 1, 1)
permutations = list(itertools.product(l_cars, l_moto))
len(permutations)

49

In [8]:
permutations = [(12, 6)]

In [9]:
run = True
# TODO: update code to match function in new version
epochs = 32
n_jobs = epochs
default_rng = np.random.default_rng(1024)
seeds = default_rng.integers(1e8, size=epochs * len(permutations))
if run:
    with Parallel(n_jobs=n_jobs, prefer="processes") as master:
        for n, permutation in tqdm(enumerate(permutations)):
            start = n * epochs
            end = (n + 1) * epochs
            items = master(delayed(batch)(seed, permutation) for seed in seeds[start:end])
            # Dump to JSONL https://jsonlines.org/examples/
            # https://stackoverflow.com/questions/38915183/
            with open(f"{path}{permutation}.jsonl", "w") as outfile:
                for item in items:
                    json.dump(item, outfile)
                    outfile.write("\n")
                # json.dump(items, outfile)
            # Compress as Zip and delete original JSON
            with zipfile.ZipFile(f"{path}{permutation}.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
                zipdir(path, zipf)
            print(permutation)
            items = None

0it [00:00, ?it/s]

(12, 6)
